<a href="https://colab.research.google.com/github/pavankumarallu/K_ML/blob/main/search_Auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, LeakyReLU, BatchNormalization, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

import numpy as np
from PIL import Image
import os 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.client import device_lib

from sklearn.cluster import KMeans
from skimage.color import rgb2hed
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
trainData = "/content/drive/MyDrive/img"

In [4]:
new_train = []


for filename in os.listdir(trainData):
    if filename.endswith('.jpg'):
        image = cv2.imread(os.path.join(trainData, filename), cv2.IMREAD_UNCHANGED)
        resized_image = cv2.resize(image, (128, 128)) 
        new_train.append(np.asarray( resized_image, dtype="uint8" ))


In [5]:
x_train = np.asarray(new_train)

x_train = x_train.astype('float32') / 255.

In [6]:
print(x_train.shape)


(1227, 128, 128, 3)


In [7]:
from keras.preprocessing.image import ImageDataGenerator
generated_data = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, rotation_range=0,  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip = True, vertical_flip = False)
generated_data.fit(x_train)

In [8]:
n_epochs = 20
batch_size = 32
optimizer = 'adam'
loss = 'mean_squared_error'

input_img = Input(shape=(128, 128, 3))

encoder = Conv2D(filters=16, kernel_size=(3,3),strides=1, padding='same')(input_img)
encoder = LeakyReLU()(encoder)
encoder = Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same')(encoder)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization()(encoder)
encoder = Conv2D(filters=64, kernel_size=(3,3),strides=2, padding='same')(encoder)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization()(encoder)
encoder = MaxPooling2D()(encoder)
encoder = Conv2D(filters=128, kernel_size=(3,3),strides=2, padding='same')(encoder)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization()(encoder)
encoder = MaxPooling2D()(encoder)
encoder = Conv2D(filters=64, kernel_size=(3,3),strides=1, padding='same')(encoder)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization()(encoder)
encoder = Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same')(encoder)
encoder = LeakyReLU()(encoder)


bottleneck = Conv2D(filters=16, kernel_size=(1,1),strides=1, padding='same')(encoder)
bottleneck = LeakyReLU()(bottleneck)


decoder = Conv2D(filters=32, kernel_size=(1,1),strides=1, padding='same')(bottleneck)
decoder = LeakyReLU()(decoder)
decoder = Conv2D(filters=64, kernel_size=(3,3),strides=1, padding='same')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization()(decoder)
decoder = UpSampling2D()(decoder)
decoder = Conv2DTranspose(filters=128, kernel_size=(3,3),strides=2, padding='same')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization()(decoder)
decoder = UpSampling2D()(decoder)
decoder = Conv2DTranspose(filters=64, kernel_size=(3,3),strides=2, padding='same')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization()(decoder)
decoder = Conv2D(filters=32, kernel_size=(3,3),strides=1, padding='same')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization()(decoder)
decoder = Conv2D(filters=16, kernel_size=(3,3),strides=1, padding='same')(decoder)
decoder = LeakyReLU()(decoder)
decoder = Conv2D(filters=3, kernel_size=(3,3),strides=1, padding='same')(decoder)
decoded = LeakyReLU()(decoder)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=optimizer , loss=loss)

filepath = "/February_epochs{}_batch{}_{}.h5".format(n_epochs,batch_size,optimizer)

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min', restore_best_weights=True)

callbacks_list = [checkpoint, ES]

In [9]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 16)      448       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496 

In [ ]:
autoencoder_train = autoencoder.fit(x_train, x_train,
                                        epochs=n_epochs,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        validation_split = 0.2,
                                        callbacks=callbacks_list
                                       )

Epoch 1/20
31/31 [==============================] - 154s 5s/step - loss: 0.2192 - val_loss: 0.3009

Epoch 00001: val_loss improved from inf to 0.30091, saving model to /February_epochs20_batch32_adam.h5
Epoch 2/20
31/31 [==============================] - 150s 5s/step - loss: 0.0317 - val_loss: 0.1367

Epoch 00002: val_loss improved from 0.30091 to 0.13674, saving model to /February_epochs20_batch32_adam.h5
Epoch 3/20
31/31 [==============================] - 150s 5s/step - loss: 0.0254 - val_loss: 0.0758

Epoch 00003: val_loss improved from 0.13674 to 0.07575, saving model to /February_epochs20_batch32_adam.h5
Epoch 4/20
31/31 [==============================] - 150s 5s/step - loss: 0.0215 - val_loss: 0.0460

Epoch 00004: val_loss improved from 0.07575 to 0.04599, saving model to /February_epochs20_batch32_adam.h5
Epoch 5/20
31/31 [==============================] - 149s 5s/step - loss: 0.0196 - val_loss: 0.0415

Epoch 00005: val_loss improved from 0.04599 to 0.04148, saving model to /Feb

In [ ]:
x_train.shape

In [ ]:
E_train = autoencoder.predict(x_train)
E_train_flatten = E_train.reshape((-1, np.prod((128,128,3))))

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
knn = NearestNeighbors(n_neighbors=5, metric="cosine")
knn.fit(E_train_flatten)

In [ ]:
new_test = []
test_img = cv2.imread("/content/3.jpg")
resized_image = cv2.resize(test_img, (128, 128)) 
new_test.append(np.asarray( resized_image, dtype="uint8" ))


In [ ]:
x_test = np.asarray(new_test)
x_test = np.asarray(x_test.astype('float32') / 255.)

In [ ]:
x_test.shape

In [ ]:
outDir = "/content/Output images"
if not os.path.exists(outDir):
    os.makedirs(outDir)
    

In [ ]:
E_test = autoencoder.predict(x_test)
E_test_flatten = E_test.reshape((-1, np.prod(( 128,128,3))))

In [ ]:
E_test.shape,E_train.shape,E_test_flatten.shape,E_train_flatten.shape

In [ ]:
_,indeces = knn.kneighbors(E_test_flatten)

In [ ]:
indeces

In [ ]:
lis = []
for i in indeces:
  for j in i:
    lis.append(j)

In [ ]:
rows = 9
columns = 9

In [ ]:
fig = plt.figure(figsize=(10, 7))

In [ ]:
for i in range(len(lis)):
  fig.add_subplot(rows, columns, i+1)
  plt.imshow(new_train[lis[i]])
  plt.axis('off')
  plt.title(str(i))
fig.show()